## Pre-Processing_Dataset_2.py
Alfredo Andere

Input: E’ - mne data structure. <p>
Output: E - Dataset of epochs of 30 seconds of filtered and normalized data

In [1]:
import os
import numpy as np
import pandas as pd
import mne

In [2]:
### Toy data to work with
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis_filt-0-40_raw.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file)

Opening raw data file /Users/alfredoanderejr/mne_data/MNE-sample-data/MEG/sample/sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Current compensation grade : 0


#### Divide into 30s samples
Input: E’ - a mne data structure of n number of recordings and t seconds each. <p>
Output: E - dataset of epochs e of 30 seconds but temporally ordered. 
        #epochs = |E’| = (n * t) / 30

In [ ]:
def _create_events(raw, epoch_length):
    """Create events to split raw into epochs."""
    file_length = raw.n_times
    first_samp = raw.first_samp
    sfreq = raw.info['sfreq']
    n_samp_in_epoch = int(epoch_length * sfreq)

    n_epochs = int(file_length // n_samp_in_epoch)

    events = []
    for i_epoch in range(n_epochs):
        events.append([first_samp + i_epoch * n_samp_in_epoch, int(0), int(0)])
    events = np.array(events)
    return events


def divide_epochs(E, e_len):
    """ Divide"""
    if raw.times[-1] >= ep_length:
        events = _create_events(raw, ep_length)
    epochs = Epochs(raw, events=events, tmin=0, tmax=e_len) 
    return epochs

#### Downsample to 128 Hz and 3 channels 	[for MASS = 256 Hz]
Input: E’  - a mne data structure sampled at a rate r’ > 128 Hz <p>
Output: E’ - a mne data structure sampled at a rate r of 128 Hz. 

In [ ]:
def downsample(epochs, chs, Hz=128):
    """ Downsample the EEG epoch to Hz=128 Hz and to only 
        include the channels in ch.
    
        Args: 
            E: mne data structure sampled at a rate r’ > 128 Hz
            channels: list of the channels to keep
            Hz: Hz to downsample to (default 128 Hz)
        Returns
            E: a mne data structure sampled at a rate r of 128 Hz.
    """
    ch_id = [E.ch_names.index(ch) for ch in chs]
    E = E.pick_types(eeg=True, selection=ch_id)
    E = E.resample(128, npad='auto')
    return E

#### Normalization by sample 
Input: E’ - dataset (ds) of mne epoch objects eE’
Output: E  ds of mne epoch objects normalized s.t <p>
        mean(e)= 0, var(e) = 1 eE <p>
Return E - Dataset of labeled e 

#### Return E - Dataset of labeled e 